背景说明：泰坦尼克沉船是震惊世界的海难事件，1912年4月15日，在它的处女航中，撞上冰川后沉没。造成了超过1502人死亡，该事件也引起了全世界对于船舶安全法规的重视。在这场灾难中，有一些因素也导致了部分乘客的获救机率比较高，如老人，小孩，上流阶层，我们的目标是利用机器学习算法对获救乘客就行准确的预测。

该数据集由两部分组成：
* 训练集：891条
* 测试集：418条

### 数据特征说明

字段           含义
_________________________
* PassengerID: 乘客编号
* Survived：是否获救（获救：1，没有获救：0）
* Pclass：船舱等级
* Name：姓名
* Sex：性别
* Age：年龄
* SibSp：兄弟姐妹和妻子的数量
* Parch：孩子和老人的数量
* Ticket：船票编号
* Fare：船票价格
* Cabin：船舱编号
* Embarked：登陆港口

### 数据处理

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_csv('D:\\Py_dataset\\train.csv')
df_test = pd.read_csv('D:\\Py_dataset\\test.csv')
print(df_train.shape,df_test.shape)

(891, 12) (418, 11)


In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

可以看出"Age","Cabin","Embarked"三个特征存在缺失值。

In [7]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 特征选取

不做复杂的特征工程，采用最快速的方法做一个预测。

**数据空值处理**
* 1.Cabin列缺失值数量较多，直接填充会对最终结果产生较大的误差影响，暂时不考虑该特征。
* 2.Age列对最终结果的影响较大，取Age的中位数对空值进行填充。
* 3.PassengerID为连续的序列值，对最终结果没有影响，不考虑该特征。

In [10]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


#### 线性回归算法

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

#选取简单的可用特征
features = ['Pclass','Age','SibSp','Parch','Fare']


LR = LinearRegression()
#样本分成三折交叉验证
kf = KFold(n_splits = 3,shuffle = False)
predictions = []
for train_index,test_index in kf.split(df_train):
    train_predictors = df_train[features].iloc[train_index,:]
    train_target = df_train['Survived'].iloc[train_index]
    LR.fit(train_predictors,train_target)
    test_predictions = LR.predict(df_train[features].iloc[test_index,:])
    predictions.append(test_predictions)


In [17]:
predictions = np.concatenate(predictions,axis = 0)

In [19]:
predictions[predictions > 0.5] = 1
predictions[predictions <= 0.5] = 0

accuracy = sum(predictions == df_train['Survived'])/len(predictions)

print('accuracy:',accuracy)

accuracy: 0.7037037037037037


#### 逻辑回归算法

In [22]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

Lr = LogisticRegression()

score = cross_val_score(Lr,df_train[features],df_train['Survived'],cv=3)
print(score.mean())

0.6992143658810326


C:\Users\taon1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\taon1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\taon1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**增加"Sex","Embarked"特征**

In [23]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
sex_map = {'male':0,'female':1}
df_train['Sex'] = df_train['Sex'].map(sex_map)

In [25]:
df_train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [26]:
df_train['Embarked'] = df_train['Embarked'].fillna('S')

In [27]:
embarked_map = {'S':0,'C':1,'Q':2}
df_train['Embarked'] = df_train['Embarked'].map(embarked_map)

In [28]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0


In [29]:
features = ['Pclass','Age','SibSp','Parch','Fare','Sex','Embarked']

score = cross_val_score(Lr,df_train[features],df_train['Survived'],cv=3)
print(score.mean())

0.7878787878787877


C:\Users\taon1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\taon1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\taon1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


从上述结果可以看出，增加新的'Sex','Embarked'特征，模型效果有一个极大的提升。

#### 随机森林算法

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

RFC = RandomForestClassifier(n_estimators = 10,min_samples_split = 2,min_samples_leaf = 1)

kf = KFold(n_splits = 3)

scores = cross_val_score(RFC,df_train[features],df_train['Survived'],cv = kf)
print(scores.mean())

0.7957351290684623


#### 使用网格搜索寻找最佳参数组合

In [34]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators':[10,20,30,40,50,60,70,80,90,100],'max_depth':[2,3,4,5,6,7,8,9]}

grid = GridSearchCV(RFC,param_grid = param_grid,scoring = 'roc_auc',cv = 5)
grid.fit(df_train[features],df_train['Survived'])

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'max_depth': [2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [37]:
print(grid.best_params_,grid.best_score_)

{'max_depth': 7, 'n_estimators': 40} 0.8709830191144069


In [39]:
#选择最佳的模型

RFC = RandomForestClassifier(n_estimators = 40,max_depth = 7)

#### 对测试集数据进行处理

In [44]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [45]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [46]:
# 对df_test的缺失数据集进行处理

df_test['Age'] = df_test['Age'].fillna(df_test['Age'].median())
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].max())

In [47]:
sex_map = {'male':0,'female':1}
df_test['Sex'] = df_test['Sex'].map(sex_map)
embarked_map = {'S':0,'C':1,'Q':2}
df_test['Embarked'] = df_test['Embarked'].map(embarked_map)

In [48]:
# 对RFC模型进行训练

RFC.fit(df_train[features],df_train['Survived'])
prediction = RFC.predict(df_test[features])
prediction[:10]

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0], dtype=int64)